<a href="https://colab.research.google.com/github/Tirumalarao047/AI-Projets/blob/main/Netflix_Recommendor_System(KMeans%2BWord2Vec%2BBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%cd /kaggle/input/netflix-movies-and-tv-shows
df = pd.read_csv('netflix_titles.csv')

In [ ]:
df.head()

In [ ]:
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
print(df.info())

In [ ]:
def analyse_missing_value(df):
    missing_value = df.isnull().sum()
    missing_percentage = (missing_value/len(df))*100
    missing_df = pd.DataFrame({
        'Missing Value': missing_value,
        'Percentage': missing_percentage.round(2)
    })
    print("\nMissing Value Analysis:")
    print(missing_df[missing_df['Missing Value']>0].sort_values('Missing Value', ascending = False))

analyse_missing_value(df)
print("\n=============================================")
print("\nContent Type Distribution:")
print(df['type'].value_counts())

In [ ]:
#Release Year Analysis
plt.figure(figsize = (15,6))
df['release_year'].value_counts().sort_index().plot(kind='bar')
plt.title('Content Release Year Distribution')
plt.xlabel('Release Year')
plt.ylabel('Number of Titles')
plt.grid(True)
plt.show()


#Rating Distribution
plt.figure(figsize = (15,6))
df['rating'].value_counts().plot(kind = "bar")
plt.title('Content Rating Distribution')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Top 10 Countries
def analyze_countries(df):
    # Split multiple countries and count each occurrence
    countries = df['country'].dropna().str.split(', ').explode()
    top_countries = countries.value_counts().head(10)

    plt.figure(figsize=(12, 6))
    top_countries.plot(kind='bar')
    plt.title('Top 10 Countries Producing Content')
    plt.xlabel('Country')
    plt.ylabel('Number of Titles')
    plt.xticks(rotation=45)
    plt.show()

analyze_countries(df)

# Content Categories Analysis
def analyze_categories(df):
    categories = df['listed_in'].dropna().str.split(', ').explode()
    top_categories = categories.value_counts().head(15)

    plt.figure(figsize=(15, 6))
    top_categories.plot(kind='bar')
    plt.title('Top 15 Content Categories')
    plt.xlabel('Category')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

analyze_categories(df)


In [ ]:
def analyze_duration(df):
    # Separate movies and TV shows
    movies = df[df['type'] == 'Movie'].copy()  # Use .copy() to avoid SettingWithCopyWarning
    tv_shows = df[df['type'] == 'TV Show'].copy()

    # Convert movie duration to numeric (remove 'min' and convert to int)
    movies['duration_num'] = movies['duration'].str.extract('(\d+)').astype(float)

    # Plot movie duration distribution
    plt.figure(figsize=(12, 6))
    sns.histplot(data=movies, x='duration_num', bins=50)
    plt.title('Movie Duration Distribution')
    plt.xlabel('Duration (minutes)')
    plt.ylabel('Count')
    plt.show()

    # Analyze TV show seasons
    print("\nTV Show Seasons Distribution:")
    # Extract numbers from strings like "2 Seasons" using regex
    tv_seasons = tv_shows['duration'].str.extract('(\d+)').astype(float).squeeze()
    season_distribution = tv_seasons.value_counts().sort_index()
    print(season_distribution)

    # Plot TV show seasons distribution
    plt.figure(figsize=(10, 6))
    season_distribution.plot(kind='bar')
    plt.title('TV Show Seasons Distribution')
    plt.xlabel('Number of Seasons')
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.show()

analyze_duration(df)

In [ ]:
def analyze_addition_trends(df):
    # Create a copy of the dataframe
    df_copy = df.copy()

    # Clean date_added column by removing leading/trailing spaces
    df_copy['date_added'] = df_copy['date_added'].str.strip()

    # Convert date_added to datetime with mixed format
    df_copy['date_added'] = pd.to_datetime(df_copy['date_added'], format='mixed')

    # Group by month and year
    monthly_additions = df_copy.groupby(df_copy['date_added'].dt.to_period('M')).size()

    # Create the plot
    plt.figure(figsize=(15, 6))
    monthly_additions.plot(kind='line', marker='o', markersize=4)
    plt.title('Content Addition Trends Over Time')
    plt.xlabel('Date')
    plt.ylabel('Number of Titles Added')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print some statistics about content additions
    print("\nContent Addition Statistics:")
    print(f"Peak month: {monthly_additions.idxmax()} with {monthly_additions.max()} titles")
    print(f"Average monthly additions: {monthly_additions.mean():.1f} titles")

def analyze_directors(df):
    # Get top directors by number of titles
    top_directors = df['director'].dropna().str.split(', ').explode().value_counts().head(10)

    # Create a more visually appealing plot
    plt.figure(figsize=(12, 6))
    colors = plt.cm.viridis(np.linspace(0, 0.8, len(top_directors)))
    bars = top_directors.plot(kind='bar', color=colors)
    plt.title('Top 10 Directors by Number of Titles', pad=20)
    plt.xlabel('Director')
    plt.ylabel('Number of Titles')
    plt.xticks(rotation=45, ha='right')

    # Add value labels on top of bars
    for i, v in enumerate(top_directors):
        plt.text(i, v, str(v), ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

    # Print additional director statistics
    print("\nDirector Statistics:")
    print(f"Total number of unique directors: {df['director'].dropna().str.split(', ').explode().nunique()}")
    print("\nTop 10 Directors and their title counts:")
    for director, count in top_directors.items():
        print(f"{director}: {count} titles")

# Function to show more detailed summary statistics
def show_summary_statistics(df):
    print("\nSummary Statistics for Numeric Columns:")

    # Basic statistics for release_year
    print("\nRelease Year Statistics:")
    print(df['release_year'].describe())

    # Content type distribution
    print("\nContent Type Distribution:")
    print(df['type'].value_counts())

    # Rating distribution
    print("\nRating Distribution:")
    print(df['rating'].value_counts())

    # Calculate percentage of missing values
    missing_data = (df.isnull().sum() / len(df)) * 100
    print("\nPercentage of Missing Values:")
    print(missing_data[missing_data > 0].round(2))

# Run the analyses
analyze_addition_trends(df)
analyze_directors(df)
show_summary_statistics(df)

In [ ]:
print("\nUnique Values in Each Column:")
for column in df.columns:
    print(f"\n{column}: {df[column].nunique()} unique values")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import re
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import tensorflow as tf
import tensorflow_hub as hub
from scipy.sparse import hstack

In [ ]:
# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
class NetflixRecommender:
    def __init__(self, df):
        self.df = df.copy()
        self.df['description'] = self.df['description'].fillna('')
        self.df['director'] = self.df['director'].fillna('')
        self.df['cast'] = self.df['cast'].fillna('')
        self.df['listed_in'] = self.df['listed_in'].fillna('')

        # Create a combined features column
        self.df['combined_features'] = self.create_combined_features()

        # Create TF-IDF matrix
        self.tfidf = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.tfidf.fit_transform(self.df['combined_features'])

        # Calculate similarity matrix
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)

    def clean_text(self, text):
        """Clean and preprocess text data"""
        # Convert to lowercase and remove special characters
        text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        words = text.split()
        words = [w for w in words if w not in stop_words]

        return ' '.join(words)

    def create_combined_features(self):
        """Combine different features into a single string"""
        return (self.df['description'].apply(self.clean_text) + ' ' +
                self.df['director'].apply(self.clean_text) + ' ' +
                self.df['cast'].apply(self.clean_text) + ' ' +
                self.df['listed_in'].apply(self.clean_text))

    def get_recommendations(self, title, n_recommendations=5):
        """Get n_recommendations similar titles based on the input title"""
        try:
            # Find the index of the movie/show
            idx = self.df[self.df['title'] == title].index[0]

            # Get similarity scores for all titles
            similarity_scores = list(enumerate(self.similarity_matrix[idx]))

            # Sort based on similarity scores
            similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

            # Get top n similar titles (excluding the input title)
            similar_titles = similarity_scores[1:n_recommendations+1]

            recommendations = []
            for i, score in similar_titles:
                recommendations.append({
                    'title': self.df.iloc[i]['title'],
                    'type': self.df.iloc[i]['type'],
                    'description': self.df.iloc[i]['description'],
                    'similarity_score': score,
                    'genre': self.df.iloc[i]['listed_in']
                })

            return recommendations

        except IndexError:
            return f"Title '{title}' not found in the dataset."


In [ ]:
df = pd.read_csv('netflix_titles.csv')

# Initialize recommender
recommender = NetflixRecommender(df)

# Example: Get recommendations for a specific title
test_title = "Stranger Things"
recommendations = recommender.get_recommendations(test_title)

# Print recommendations
print(f"\nRecommendations based on '{test_title}':")
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec['title']}")
    print(f"Type: {rec['type']}")
    print(f"Genre: {rec['genre']}")
    print(f"Similarity Score: {rec['similarity_score']:.2f}")
    print(f"Description: {rec['description'][:100]}...")


In [ ]:
class AdvancedNetflixRecommender:
    def __init__(self, df):
        self.df = df.copy()
        # Fill NaN values
        self.df['description'] = self.df['description'].fillna('')
        self.df['director'] = self.df['director'].fillna('')
        self.df['cast'] = self.df['cast'].fillna('')
        self.df['listed_in'] = self.df['listed_in'].fillna('')

        # Download required NLTK data
        nltk.download('stopwords')
        nltk.download('punkt')

        # Initialize models
        self.bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        self.init_models()

    def clean_text(self, text):
        """Clean and preprocess text"""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        tokens = [t for t in tokens if t not in stop_words]
        return ' '.join(tokens)

    def init_models(self):
        """Initialize all component models"""
        # Process descriptions with BERT
        print("Processing descriptions with BERT...")
        self.description_embeddings = self.bert_model.encode(self.df['description'].tolist())

        # Create TF-IDF vectors for different features
        print("Creating TF-IDF vectors...")
        self.tfidf = TfidfVectorizer(stop_words='english')
        self.genre_matrix = self.tfidf.fit_transform(self.df['listed_in'])

        # Train Word2Vec on descriptions
        print("Training Word2Vec model...")
        descriptions_tokens = [word_tokenize(self.clean_text(desc))
                             for desc in self.df['description']]
        self.word2vec_model = Word2Vec(sentences=descriptions_tokens,
                                     vector_size=1000,
                                     window=10,
                                     min_count=1)

        # Create description embeddings using Word2Vec
        self.desc_w2v_embeddings = np.array([
            np.mean([self.word2vec_model.wv[word]
                    for word in word_tokenize(self.clean_text(desc))
                    if word in self.word2vec_model.wv]
                   or [np.zeros(1000)], axis=0)
            for desc in self.df['description']
        ])

        # Cluster similar content
        print("Clustering content...")
        self.n_clusters = 10
        combined_features = np.concatenate([
            self.description_embeddings,
            self.desc_w2v_embeddings
        ], axis=1)

        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
        self.clusters = self.kmeans.fit_predict(combined_features)
        self.df['cluster'] = self.clusters

    def get_description_similarity(self, idx):
        """Get similarity scores based on BERT description embeddings"""
        return cosine_similarity(
            self.description_embeddings[idx].reshape(1, -1),
            self.description_embeddings
        ).flatten()

    def get_genre_similarity(self, idx):
        """Get similarity scores based on genre TF-IDF"""
        return cosine_similarity(
            self.genre_matrix[idx],
            self.genre_matrix
        ).flatten()

    def get_w2v_similarity(self, idx):
        """Get similarity scores based on Word2Vec embeddings"""
        return cosine_similarity(
            self.desc_w2v_embeddings[idx].reshape(1, -1),
            self.desc_w2v_embeddings
        ).flatten()

    def get_recommendations(self, title, n_recommendations=5, weights=None):
        """
        Get recommendations using weighted combination of different similarity metrics

        Parameters:
        - title: str, title to base recommendations on
        - n_recommendations: int, number of recommendations to return
        - weights: dict, weights for different similarity components
                  (default: {'description': 0.4, 'genre': 0.3, 'w2v': 0.3})
        """
        try:
            # Default weights if none provided
            if weights is None:
                weights = {
                    'description': 0.4,
                    'genre': 0.3,
                    'w2v': 0.3
                }

            # Find index of the title
            idx = self.df[self.df['title'] == title].index[0]
            cluster = self.clusters[idx]

            # Get similarity scores from different components
            desc_similarity = self.get_description_similarity(idx)
            genre_similarity = self.get_genre_similarity(idx)
            w2v_similarity = self.get_w2v_similarity(idx)

            # Combine similarity scores with weights
            combined_similarity = (
                weights['description'] * desc_similarity +
                weights['genre'] * genre_similarity +
                weights['w2v'] * w2v_similarity
            )

            # Boost scores for items in the same cluster
            cluster_boost = 0.2
            combined_similarity[self.clusters == cluster] += cluster_boost

            # Get top similar indices (excluding the input title)
            similar_indices = combined_similarity.argsort()[::-1][1:n_recommendations+1]

            recommendations = []
            for i in similar_indices:
                recommendations.append({
                    'title': self.df.iloc[i]['title'],
                    'type': self.df.iloc[i]['type'],
                    'description': self.df.iloc[i]['description'],
                    'genre': self.df.iloc[i]['listed_in'],
                    'similarity_score': combined_similarity[i],
                    'cluster': self.clusters[i]
                })

            return recommendations

        except IndexError:
            return f"Title '{title}' not found in the dataset."

    def get_cluster_recommendations(self, cluster_id, n_recommendations=5):
        """Get top recommendations from a specific cluster"""
        cluster_items = self.df[self.df['cluster'] == cluster_id]
        return cluster_items.sample(min(n_recommendations, len(cluster_items)))

    def analyze_user_preferences(self, watched_titles):
        """Analyze user preferences based on watch history"""
        watched_indices = [self.df[self.df['title'] == title].index[0]
                          for title in watched_titles if title in self.df['title'].values]

        if not watched_indices:
            return "No valid titles found in watch history."

        # Get average embeddings for watched content
        avg_desc_embedding = np.mean(self.description_embeddings[watched_indices], axis=0)
        avg_w2v_embedding = np.mean(self.desc_w2v_embeddings[watched_indices], axis=0)

        # Find most common cluster
        preferred_cluster = self.df.iloc[watched_indices]['cluster'].mode()[0]

        # Get genre preferences
        genre_counts = (self.df.iloc[watched_indices]['listed_in']
                       .str.split(', ')
                       .explode()
                       .value_counts())

        return {
            'preferred_cluster': preferred_cluster,
            'top_genres': genre_counts.head().to_dict(),
            'avg_embeddings': {
                'description': avg_desc_embedding,
                'w2v': avg_w2v_embedding
            }
        }

In [ ]:
# # Example usage
# if __name__ == "__main__":
#     # Read the dataset
#     df = pd.read_csv('netflix_titles.csv')

#     # Initialize recommender
#     print("Initializing recommender system...")
#     recommender = AdvancedNetflixRecommender(df)

#     # Example: Get recommendations for a specific title
#     test_title = "Naruto"
#     print(f"\nGetting recommendations for '{test_title}'...")
#     recommendations = recommender.get_recommendations(test_title)

#     # Print recommendations
#     for i, rec in enumerate(recommendations, 1):
#         print(f"\n{i}. {rec['title']}")
#         print(f"Type: {rec['type']}")
#         print(f"Genre: {rec['genre']}")
#         print(f"Similarity Score: {rec['similarity_score']:.2f}")
#         print(f"Cluster: {rec['cluster']}")

#     # Example: Analyze user preferences
#     watched_titles = ["Naruto", "Black Clover", "Bleach"]
#     print("\nAnalyzing user preferences...")
#     preferences = recommender.analyze_user_preferences(watched_titles)
#     print("\nUser Preferences Analysis:")
#     print(f"Preferred Cluster: {preferences['preferred_cluster']}")
#     print("\nTop Genres:")
#     for genre, count in preferences['top_genres'].items():
#         print(f"- {genre}: {count}")

In [ ]:
df = pd.read_csv('netflix_titles.csv')

# Initialize recommender
print("Initializing recommender system...")
recommender = AdvancedNetflixRecommender(df)

# Example: Get recommendations for a specific title
test_title = "Naruto"
print(f"\nGetting recommendations for '{test_title}'...")
recommendations = recommender.get_recommendations(test_title)

# Print recommendations
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec['title']}")
    print(f"Type: {rec['type']}")
    print(f"Genre: {rec['genre']}")
    print(f"Similarity Score: {rec['similarity_score']:.2f}")
    print(f"Cluster: {rec['cluster']}")

In [ ]:
test_title = "Vincenzo"

recommendations = recommender.get_recommendations(test_title)

    # Print recommendations
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec['title']}")
    print(f"Type: {rec['type']}")
    print(f"Genre: {rec['genre']}")
    print(f"Similarity Score: {rec['similarity_score']:.2f}")
    print(f"Cluster: {rec['cluster']}")